## DATA 622 Natural Language Processing
### Homework 5

Questions - Use the following article https://www.usatoday.com/story/news/politics/2025/06/13/petehegseth-pentagon-invade-greenland-plan/84188458007/.
1. Extract and print an extractive summary using the TextRank method.
2. Extract and print an extractive summary using a frequency-based sentence scoring
method.
3. Generate and print an abstractive summary using a pretrained transformer model
(e.g., BART or T5).
4. Print a Lead-3 summary (the first three sentences of the article).
5. Print a manual compression summary, limiting the result to about 20% of the
original sentences.
6. Use an LLM to summary the text.

In [ ]:
pip install sumy

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 77.5 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21695 sha256=17b4d15bbda23bd95f01628ef44de49144bc1e7af2c4cba4543ca03c9705812f
  Stored in directory: /root/.cache/pip/wheels/32/99/64/59305409cacd03aa03e7bddf31a9db34b1fa7033bd41972662
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=b161beb774edbfeedff9dba6832e26ddb01cf489047905dfff3c833d4449ed4c
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built breadability docopt


In [ ]:
nltk.download('punkt_tab')
nltk.download('punkt')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import requests
import nltk
from bs4 import BeautifulSoup
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import defaultdict
import string
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import networkx as nx

In [ ]:
article_link = "https://www.usatoday.com/story/news/politics/2025/06/13/pete-hegseth-pentagon-invade-greenland-plan/84188458007/"
page_data = requests.get(article_link)
webpage_soup = BeautifulSoup(page_data.content, 'html.parser')

In [ ]:
paragraphs = webpage_soup.find_all('p')
article_text = "\n".join([para.get_text() for para in paragraphs])

In [ ]:
# 1. Extract and print an extractive summary using the TextRank method.

tokenized_sents = sent_tokenize(article_text)

tfidf_model = TfidfVectorizer()
tfidf_matrix = tfidf_model.fit_transform(tokenized_sents)

similarity_mat = (tfidf_matrix * tfidf_matrix.T).toarray()

graph_network = nx.from_numpy_array(similarity_mat)
sentence_scores = nx.pagerank(graph_network)

ranked_sents = sorted(((sentence_scores[i], sent) for i, sent in enumerate(tokenized_sents)), reverse=True)

print("TextRank Scores for Sentences:\n")
for score, sent in ranked_sents:
    print(f"- ({score:.4f}) {sent}")

TextRank Scores for Sentences:

- (0.0868) "It is not your testimony today that there are plans at the Pentagon for taking by force or invading Greenland, correct?
- (0.0839) During a March visit to Pituffik Space Base, the U.S. base on Greenland, Vice President JD Vance accused Denmark of "failing" to protect the Arctic island while downplaying Trump's threats to take it over by force.
- (0.0839) Defense Secretary Pete Hegseth said the Pentagon has plans for multiple "contingencies" in Greenland – including an invasion of the island.
- (0.0832) Asked by Republican Rep. Mike Turner at a June 12 House Armed Services Committee hearing to confirm whether there are plans to invade Greenland, Hegseth said, "The Pentagon has plans for any number of contingencies."
- (0.0795) In the latest snub to Denmark and other European allies, the Pentagon reportedly plans to move its oversight of the island from U.S. European Command to U.S. Northern Command.
- (0.0785) The island is also rich in critic

In [ ]:
# 2. Extract and print an extractive summary using a frequency-based sentence scoring method.

stopword_set = set(stopwords.words("english"))
tokens = word_tokenize(article_text.lower())
filtered_tokens = [tok for tok in tokens if tok not in stopword_set and tok not in string.punctuation]

word_freq = defaultdict(int)
for tok in filtered_tokens:
    word_freq[tok] += 1

split_sents = sent_tokenize(article_text)
sent_weights = defaultdict(int)

for sent in split_sents:
    for tok in word_tokenize(sent.lower()):
        if tok in word_freq:
            sent_weights[sent] += word_freq[tok]

print("Frequency-Based Sentence Weighting:\n")
for sent in split_sents:
    print(f"[Score: {sent_weights[sent]}] {sent.strip()}\n")

Frequency-Based Sentence Weighting:

[Score: 53] Defense Secretary Pete Hegseth said the Pentagon has plans for multiple "contingencies" in Greenland – including an invasion of the island.

[Score: 64] Asked by Republican Rep. Mike Turner at a June 12 House Armed Services Committee hearing to confirm whether there are plans to invade Greenland, Hegseth said, "The Pentagon has plans for any number of contingencies."

[Score: 35] "It is not your testimony today that there are plans at the Pentagon for taking by force or invading Greenland, correct?

[Score: 15] Because I sure as hell hope that it is not your testimony," Turner dug in.

[Score: 39] "We look forward to working with Greenland to ensure that it is secured from any potential threats," Hegseth said.

[Score: 45] President Donald Trump has declined to rule out force in his pledge to "get Greenland," although he has said it won't be necessary.

[Score: 22] He has insisted that acquiring Greenland is necessary for national securi

In [ ]:
# 3. Generate and print an abstractive summary using a pretrained transformer model (e.g., BART or T5).

from transformers import pipeline, BartTokenizer, BartForConditionalGeneration

model_id = "facebook/bart-large-cnn"
text_summarizer = pipeline("summarization", model=model_id)

bart_summary = text_summarizer(split_sents, max_length=200, min_length=10, do_sample=False)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu
Your max_length is set to 200, but your input_length is only 31. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=15)
Your max_length is set to 200, but your input_length is only 46. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)
Your max_length is set to 200, but your input_length is only 26. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)
Your max_length is set to 200, but your input_length is only 19. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...

In [ ]:
for idx, sent in enumerate(split_sents):
    print(f"{bart_summary[idx]['summary_text']}\n")

Defense Secretary Pete Hegseth said the Pentagon has plans for multiple "contingencies" in Greenland.

Hegseth: "The Pentagon has plans for any number of contingencies"

"It is not your testimony today that there are plans at the Pentagon for taking by force or invading Greenland, correct? "

"Because I sure as hell hope that it is not your testimony," Turner says.

"We look forward to working with Greenland to ensure that it is secured from any potential threats," Hegseth said.

President Donald Trump has declined to rule out force in his pledge to "get Greenland" He has said it won't be necessary.

He has insisted that acquiring Greenland is necessary for national security, citing growing Chinese and Russian influence in the region.

The island is also rich in critical minerals that the U.S. wants to challenge Chinese monopolies in some industries.

U.S. Vice President JD Vance accused Denmark of "failing" to protect Greenland. He downplayed Trump's threats to take it over by force.


In [ ]:
# 4. Print a Lead-3 summary (the first three sentences of the article).

lead_three = " ".join(split_sents[:3])

print("Lead-3 Summary:\n")
print(lead_three)

Lead-3 Summary:

Defense Secretary Pete Hegseth said the Pentagon has plans for multiple "contingencies" in Greenland – including an invasion of the island. Asked by Republican Rep. Mike Turner at a June 12 House Armed Services Committee hearing to confirm whether there are plans to invade Greenland, Hegseth said, "The Pentagon has plans for any number of contingencies." "It is not your testimony today that there are plans at the Pentagon for taking by force or invading Greenland, correct?


In [ ]:
# 5. Print a manual compression summary, limiting the result to about 20% of the original sentences.

summary_count = max(1, round(len(split_sents) * 0.2))

chosen_sents = [split_sents[0], split_sents[-4]] if summary_count >= 2 else [split_sents[0]]

compressed_summary = " ".join(chosen_sents)
print("Manual Compression Summary (≈20%):\n")
print(compressed_summary)

Manual Compression Summary (≈20%):

Defense Secretary Pete Hegseth said the Pentagon has plans for multiple "contingencies" in Greenland – including an invasion of the island. Danish leaders have pushed back.


In [ ]:
# 6. Use an LLM to summary the text

from transformers import pipeline

llm_summarizer = pipeline("summarization", model="t5-base")
t5_summary = llm_summarizer(split_sents, max_length=130, min_length=30, do_sample=False)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cpu
Your max_length is set to 130, but your input_length is only 36. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=18)
Both `max_new_tokens` (=256) and `max_length`(=130) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Your max_length is set to 130, but your input_length is only 54. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=27)
Both `max_new_tokens` (=256) and `max_length`(=130) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_class

In [ ]:
for idx, sent in enumerate(split_sents):
    print(f"{t5_summary[idx]['summary_text']}\n")

Pete Hegseth said the Pentagon has plans for multiple "contingencies" in Greenland – including an invasion of the island .

hegseth: "the Pentagon has plans for any number of contingencies" he was asked if there are plans to invade greenland .

"it is not your testimony today that there are plans at the Pentagon for taking by force or invading Greenland, correct?

"I sure as hell hope that it is not your testimony," Turner says . he says he doesn't know if he'll be able to get a transcript .

"we look forward to working with Greenland to ensure that it is secured from any potential threats," says Hegseth .

president Donald Trump has declined to rule out force in his pledge to "get Greenland" he has said it won't be necessary .

he insists that acquiring Greenland is necessary for national security . citing growing Chinese and Russian influence in the region .

the island is also rich in critical minerals that the U.S. wants to challenge monopolies in some industries . u.s. wants a cha